In [1]:
import torch; 
print(torch.__version__)
print(torch.version.cuda)

1.6.0
10.2


In [3]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.6.0+cu102.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.6.0+cu102.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.6.0+cu102.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.6.0+cu102.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0+cu102.html
  Using cached https://pytorch-geometric.com/whl/torch-1.6.0%2Bcu102/torch_scatter-2.0.6-cp37-cp37m-linux_x86_64.whl (2.9 MB)
Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0+cu102.html
  Using cached https://pytorch-geometric.com/whl/torch-1.6.0%2Bcu102/torch_sparse-0.6.9-cp37-cp37m-linux_x86_64.whl (1.6 MB)
Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0+cu102.html
  Using cached https://pytorch-geometric.com/whl/torch-1.6.0%2Bcu102/torch_cluster-1.5.9-cp37-cp37m-linux_x86_64.whl (1.1 MB)
Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0+cu102.html
  Using cached https://pytorch-geometric.com/whl/torch-1.6.0%2Bcu102/torch_spline_conv-1.2.1-cp37-cp37m-linux_x86_64.whl (371 kB)
  Using cached torch_geometric-1.7.0-py3-none-any.whl
  Using cached ase-3.21.1-py3-none-any.whl (2.2 MB)
  Using cached python_louvain-0.15-py3-none-any.whl
  Using cached networkx-2.5.1-py3-n

In [17]:
import os.path as osp
import argparse

import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from tqdm import tqdm
from IPython.core.debugger import set_trace

import json
import pandas as pd
import numpy as np

In [2]:
def edge_pred_prep(data):
    E = data.edge_index.T.tolist()
    size = len(set([n for e in E for n in e])) 
    adjacency = [[0]*size for _ in range(size)]
    for sink, source in E:
        adjacency[sink][source] = 1

    yy = []
    for i in range(len(data.x)):
        for j in range(i, len(data.x)):
              yy.append(adjacency[i][j])

    y_edge = torch.Tensor(yy).int()
    data.y = y_edge.long()
    
    mask = [False for _ in range(len(y_edge))]
    train_mask = mask.copy()
    val_mask = mask.copy()
    test_mask = mask.copy()
    
    for i in range(int(len(y_edge)*1/100)):
        train_mask[i] = True

    for i in range(int(len(y_edge)*1/100), int(len(y_edge)*2/3)):
        val_mask[i] = True

    for i in range(int(len(y_edge)*1/100), len(y_edge)):
        test_mask[i] = True
        
    data.train_mask = torch.Tensor(train_mask).bool()
    data.val_mask = torch.Tensor(val_mask).bool()
    data.test_mask = torch.Tensor(test_mask).bool()

In [121]:
SENTENSES = pd.read_csv("pulse_processed_test.csv", index_col = 0 )["message"]

In [122]:
SENTENSES = SENTENSES.values.tolist()

In [125]:
SENTENSES

[['а',
  'я',
  'и',
  'не',
  'выступать',
  'он',
  'я',
  'тоже',
  'не',
  'особо',
  'симпатичный',
  'предыстория',
  'почитать'],
 ['хотеть',
  'рассказывать',
  'как',
  'заинтриговывать',
  'человек',
  'завтра',
  'рассказывать',
  'принятие',
  'что'],
 ['инвестинг', 'com'],
 ['ересь', 'смотреть', 'хотя', 'бы', 'канал', 'силиконовый', 'долина'],
 ['я',
  'тинькоф',
  'кинуть',
  'с',
  'плечо',
  'самый',
  'дорогой',
  'урок',
  'в',
  'жизнь',
  'когда',
  'вырастать',
  'и',
  'когда',
  'падать',
  'приложение',
  'мороснуть',
  'не',
  'смочь',
  'продавать',
  'акция',
  'а',
  'когда',
  'совсем',
  'упасть',
  'сам',
  'продавать',
  'маржа',
  'кол',
  'не',
  'доверять',
  'приложение',
  'на',
  'процент'],
 ['смотреть', 'тоже', 'надеяться', 'на', 'сбор'],
 ['быть',
  'и',
  'плюс',
  'опыт',
  'и',
  'нафиг',
  'стоплосс',
  'акция',
  'стабильный',
  'компания',
  'все',
  'равный',
  'вырастать',
  'терпение'],
 ['что', 'торговать', 'можно', 'до', 'декабрь', 'н

In [124]:
for i in range(len(SENTENSES)):
    SENTENSES[i] = str(SENTENSES[i]).split("   ")

In [ ]:
 SENTENSES[][0][0].split("   ")

In [49]:
dvc = 'cuda:0'#'cpu'

In [138]:
test_mask = torch.Tensor(test_mask).bool()

In [15]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(20833, 16, cached=True,
                             normalize=True)
        self.conv2 = GCNConv(16, 2, cached=True,
                             normalize=True)

    def forward(self):
        
        def doc_embedding(x):
            doc_emb = torch.ones([len(SENTENSES), 2], device=torch.device(dvc))
            for i in tqdm(range(len(SENTENSES))):
                for j in range(len(SENTENSES[i][0])):
#                    print(len(x[SENTENSES[i][0][j]]))
#                    print(len(doc_embedding[i]))
                    if SENTENSES[i][0][j] in word_id:
                        doc_emb[i] *= x[word_id[SENTENSES[i][0][j]]]
            return doc_emb
                
        
        def edge_embedding(x):
            index = 0
            edge_embeddings = torch.empty([3667986, 2], device=torch.device(dvc))
            print(len(x))
            for i in tqdm(range(len(x))):
                for j in range(i, len(x)):
                    
                    edge_embeddings[index] = torch.mul(x[i], x[j])
                    index += 1
            return edge_embeddings

        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr
        x = F.relu(self.conv1(x, edge_index, None))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, None)
        print(len(x))
        print(len(x[0]))
        edge_x = doc_embedding(x)
        print("OUTPUT LEN - " + str(len(edge_x)))
        return F.log_softmax(edge_x, dim=1)

def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()
    return model()


from sklearn.metrics import f1_score

@torch.no_grad()
def test():
    model.eval()
    logits, accs = model(), []
    result, f1s = [], []
    for _, mask in data('train_mask', 'test_mask'):
        
        pred = logits[mask].max(1)[1]
        result.append(logits[mask])
#       print(logits[mask].max(1))
#        print(mask)
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
        f1s.append([f1_score(data.y[mask].cpu(), pred.cpu(), average='macro'),
                    f1_score(data.y[mask].cpu(), pred.cpu(), average='micro'),
                    f1_score(data.y[mask].cpu(), pred.cpu(), average='weighted'),
                    f1_score(data.y[mask].cpu(), pred.cpu(), average=None)])
    return [accs,  f1s, result]




In [50]:
with open("id_word_small.json", "r") as read_file:
    id_word = json.load(read_file)

In [51]:
with open("word_id_small.json", "r") as read_file:
    word_id = json.load(read_file)

In [52]:
word_id[SENTENSES[1][0][2]]

18747

In [22]:
len(SENTENSES)

390021

In [23]:
len(id_word)

20833

In [31]:
EDGES = pd.read_csv("edges_small.txt", names=["from", "to"], sep= " ")

In [32]:
len(EDGES)

695305

In [33]:
EDGES[EDGES["from"] == 6]

,from,to
589860,6,9991
589861,6,14421
589862,6,140
589863,6,8866
589864,6,18584
589865,6,15918
589866,6,6340
589867,6,943
589868,6,12623
589869,6,16495


In [34]:
edge_index = torch.tensor(np.array(EDGES).T, dtype=torch.long)

In [35]:
y = torch.tensor(pd.read_csv("processed_external_new.csv", index_col = 0)['y'].to_numpy(), dtype=torch.long)

In [129]:
len(SENTENSES)

6170

In [136]:
train_mask = []
test_mask = []
for i in range(len(SENTENSES)):
    test_mask.append(False)
for i in range(len(SENTENSES)):
    test_mask[i] = True

In [131]:
test_mask 

tensor([False, False, False,  ...,  True,  True,  True])

In [57]:
x = np.identity(len(id_word))

In [58]:
pmi = pd.read_csv("pmi_small.txt", header=None)

In [59]:
pmi

,0
0,9.901965
1,4.772066
2,7.624698
3,7.145125
4,2.412902
...,...
695300,4.519027
695301,7.669649
695302,5.279446
695303,3.011211


In [60]:
edge_attr = torch.tensor(np.array(pmi).T, dtype=torch.float)

In [139]:
from torch_geometric.data import Data
x = torch.tensor(x, dtype=torch.float)
data = Data(x=x, edge_index=edge_index, train_mask = train_mask, test_mask = test_mask, edge_attr = edge_attr)

/home/user/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [140]:
data

Data(edge_attr=[1, 695305], edge_index=[2, 695305], test_mask=[6170], train_mask=[0], x=[20833, 20833])

In [126]:
data

Data(edge_attr=[1, 695305], edge_index=[2, 695305], test_mask=[390021], train_mask=[390021], x=[20833, 20833], y=[390021])

In [96]:
data

Data(edge_attr=[1, 4902478], edge_index=[2, 4902478], test_mask=[210738], train_mask=[210738], x=[210738, 210738], y=[390021])

In [14]:
data

Data(edge_index=[2, 10556], test_mask=[3667986], train_mask=[3667986], val_mask=[3667986], x=[2708, 1433], y=[3667986])

In [56]:
model

Net(
  (conv1): GCNConv(20833, 16)
  (conv2): GCNConv(16, 2)
)

In [81]:
195010*2

390020

In [63]:
device = torch.device(dvc)
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters(), weight_decay=5e-4),
    dict(params=model.conv2.parameters(), weight_decay=0)
], lr=0.01)  # Only perform weight-decay on first convolution.


In [66]:
output = 0
best_val_acc = test_acc = 0
for epoch in range(1, 2):
    A = train()
    B = test()
    train_acc, tmp_test_acc = B[0]
    train_f1s, test_f1s = B[1]
    #    if val_acc > best_val_acc

    #        best_val_acc = val_acc
    test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Accuracy train: {:.4f}, Accuracy test: {:.4f}'
    print(log.format(epoch, train_acc, test_acc))
    print("F1 macro train: " + str(train_f1s[0]) + ". " + "F1 micro train: " + str(train_f1s[1]) + ". " +
         "F1 weighted train: " + str(train_f1s[2]) + ". " + "F1 nonweighted train: " + str(train_f1s[3]) + ". ")
    print("F1 macro test: " + str(test_f1s[0]) + ". " + "F1 micro test: " + str(test_f1s[1]) + ". " +
         "F1 weighted test: " + str(test_f1s[2]) + ". " + "F1 nonweighted test: " + str(test_f1s[3]) + ". ") 

  0%|          | 130/390021 [00:00<05:00, 1298.36it/s]

20833
2


100%|██████████| 390021/390021 [03:31<00:00, 1842.60it/s]


OUTPUT LEN - 390021


  0%|          | 0/390021 [00:00<?, ?it/s]

20833
2


100%|██████████| 390021/390021 [03:14<00:00, 2002.91it/s]


OUTPUT LEN - 390021
20833
2


100%|██████████| 390021/390021 [01:03<00:00, 6130.81it/s]


OUTPUT LEN - 390021
Epoch: 001, Accuracy train: 0.3205, Accuracy test: 0.5236
F1 macro train: 0.3163760632505417. F1 micro train: 0.3205271524537203. F1 weighted train: 0.3524880261554774. F1 nonweighted train: [0.36964692 0.26310521]. 
F1 macro test: 0.4005407264209122. F1 micro test: 0.5235858490033896. F1 weighted test: 0.44069880170773107. F1 nonweighted test: [0.12895181 0.67212964]. 


In [141]:
model = torch.load("/home/jovyan/text/entire")
model.eval()

RuntimeError: [enforce fail at inline_container.cc:209] . file not found: archive/data.pkl

In [142]:
model = torch.load("/home/jovyan/text/comm_gcn")

In [143]:
device = torch.device(dvc)
data = data.to(device)

In [144]:
@torch.no_grad()
def test_test():
    model.eval()
#    test_mask = torch.tensor(test_mask, dtype=torch.bool)
    mask = test_mask
    logits, accs = model(), []
    print(logits)
    pred = logits[mask].max(1)[1]
    
    return pred

In [145]:
C = test_test()

 29%|██▊       | 1767/6170 [00:00<00:00, 17667.66it/s]

20833
2


100%|██████████| 6170/6170 [00:00<00:00, 17041.51it/s]

OUTPUT LEN - 6170
tensor([[-0.5668, -0.8377],
        [-0.6931, -0.6931],
        [-0.6931, -0.6931],
        ...,
        [-0.6930, -0.6933],
        [-0.6943, -0.6920],
        [-0.6931, -0.6932]], device='cuda:0')


In [ ]:
train_mask = []
test_mask = []
for i in range(len(y)):
    train_mask.append(True)
    test_mask.append(False)
for i in range(int(len(y)*1/5), len(y)):
    train_mask[i] = False
    test_mask[i] = True
    
train_mask = torch.tensor(train_mask, dtype=torch.bool)
test_mask = torch.tensor(test_mask, dtype=torch.bool)

In [75]:
len(test_mask)*4/5

312016.8

In [151]:
df = pd.read_csv("pulse_processed_test.csv", index_col = 0 )
df['toxic'] = np.array(C.cpu())
df.to_csv("result.csv")

In [148]:
SENTENSES

[['а',
  'я',
  'и',
  'не',
  'выступать',
  'он',
  'я',
  'тоже',
  'не',
  'особо',
  'симпатичный',
  'предыстория',
  'почитать'],
 ['хотеть',
  'рассказывать',
  'как',
  'заинтриговывать',
  'человек',
  'завтра',
  'рассказывать',
  'принятие',
  'что'],
 ['инвестинг', 'com'],
 ['ересь', 'смотреть', 'хотя', 'бы', 'канал', 'силиконовый', 'долина'],
 ['я',
  'тинькоф',
  'кинуть',
  'с',
  'плечо',
  'самый',
  'дорогой',
  'урок',
  'в',
  'жизнь',
  'когда',
  'вырастать',
  'и',
  'когда',
  'падать',
  'приложение',
  'мороснуть',
  'не',
  'смочь',
  'продавать',
  'акция',
  'а',
  'когда',
  'совсем',
  'упасть',
  'сам',
  'продавать',
  'маржа',
  'кол',
  'не',
  'доверять',
  'приложение',
  'на',
  'процент'],
 ['смотреть', 'тоже', 'надеяться', 'на', 'сбор'],
 ['быть',
  'и',
  'плюс',
  'опыт',
  'и',
  'нафиг',
  'стоплосс',
  'акция',
  'стабильный',
  'компания',
  'все',
  'равный',
  'вырастать',
  'терпение'],
 ['что', 'торговать', 'можно', 'до', 'декабрь', 'н

In [147]:
pd.DataFrame(C).to_csv("")

tensor([0, 0, 1,  ..., 0, 1, 0], device='cuda:0')

In [86]:
len(B[2][0])

195010

In [65]:
torch.save(model, "/home/jovyan/text/comm_gcn")

In [52]:
train_acc, tmp_test_acc = B[0]
train_f1s, test_f1s = B[1]
#    if val_acc > best_val_acc

#        best_val_acc = val_acc
test_acc = tmp_test_acc
log = 'Epoch: {:03d}, Accuracy train: {:.4f}, Accuracy test: {:.4f}'
print(log.format(epoch, train_acc, test_acc))
print("F1 macro train: " + str(train_f1s[0]) + ". " + "F1 micro train: " + str(train_f1s[1]) + ". " +
     "F1 weighted train: " + str(train_f1s[2]) + ". " + "F1 nonweighted train: " + str(train_f1s[3]) + ". ")
print("F1 macro test: " + str(test_f1s[0]) + ". " + "F1 micro test: " + str(test_f1s[1]) + ". " +
     "F1 weighted test: " + str(test_f1s[2]) + ". " + "F1 nonweighted test: " + str(test_f1s[3]) + ". ")


Epoch: 001, Accuracy train: 0.8339, Accuracy test: 0.4263
F1 macro train: 0.4653214509844549. F1 micro train: 0.8339059535408441. F1 weighted train: 0.7662589781593674. F1 nonweighted train: [0.90925188 0.02139102]. 
F1 macro test: 0.2993087258735414. F1 micro test: 0.42629390136966633. F1 weighted test: 0.2552024171762617. F1 nonweighted test: [0.59759953 0.00101792]. 


In [79]:
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': 0.875
            }, "/home/jovyan/text/model_save")

In [130]:
a = test()



  0%|          | 0/390021 [00:00<?, ?it/s]

  0%|          | 609/390021 [00:00<01:04, 6071.78it/s]

20833
2




  0%|          | 1298/390021 [00:00<01:01, 6294.01it/s]

  0%|          | 1912/390021 [00:00<01:02, 6238.87it/s]

  1%|          | 2548/390021 [00:00<01:01, 6272.26it/s]

  1%|          | 3181/390021 [00:00<01:01, 6283.34it/s]

  1%|          | 3795/390021 [00:00<01:01, 6237.18it/s]

  1%|          | 4481/390021 [00:00<01:00, 6410.25it/s]

  1%|▏         | 5161/390021 [00:00<00:59, 6520.32it/s]

  1%|▏         | 5780/390021 [00:00<01:01, 6287.68it/s]

  2%|▏         | 6388/390021 [00:01<01:01, 6199.34it/s]

  2%|▏         | 7005/390021 [00:01<01:01, 6189.76it/s]

  2%|▏         | 7632/390021 [00:01<01:01, 6210.76it/s]

  2%|▏         | 8282/390021 [00:01<01:00, 6292.82it/s]

  2%|▏         | 8928/390021 [00:01<01:00, 6339.48it/s]

  2%|▏         | 9574/390021 [00:01<00:59, 6373.71it/s]

  3%|▎         | 10210/390021 [00:01<01:00, 6271.48it/s]

  3%|▎         | 10850/390021 [00:01<01:00, 6307.73it/s]

  3%|▎         | 11480/390021 [00:01<01:00, 6296.62it/s]

  3%|▎         | 12115/390

 40%|████      | 157354/390021 [00:26<01:16, 3058.76it/s]

 40%|████      | 157662/390021 [00:26<01:16, 3020.88it/s]

 41%|████      | 157966/390021 [00:26<01:18, 2967.54it/s]

 41%|████      | 158264/390021 [00:26<01:20, 2867.38it/s]

 41%|████      | 158563/390021 [00:27<01:19, 2903.07it/s]

 41%|████      | 158872/390021 [00:27<01:18, 2947.13it/s]

 41%|████      | 159174/390021 [00:27<01:17, 2968.38it/s]

 41%|████      | 159490/390021 [00:27<01:16, 3023.22it/s]

 41%|████      | 159794/390021 [00:27<01:18, 2933.24it/s]

 41%|████      | 160134/390021 [00:27<01:15, 3058.30it/s]

 41%|████      | 160444/390021 [00:27<01:14, 3069.62it/s]

 41%|████      | 160765/390021 [00:27<01:13, 3108.58it/s]

 41%|████▏     | 161078/390021 [00:27<01:16, 2990.25it/s]

 41%|████▏     | 161383/390021 [00:27<01:16, 3005.96it/s]

 41%|████▏     | 161685/390021 [00:28<01:18, 2900.86it/s]

 42%|████▏     | 161988/390021 [00:28<01:17, 2934.46it/s]

 42%|████▏     | 162283/390021 [00:28<01:18, 2905.96it/s

 79%|███████▉  | 308628/390021 [00:52<00:12, 6384.70it/s]

 79%|███████▉  | 309282/390021 [00:52<00:12, 6428.23it/s]

 79%|███████▉  | 309937/390021 [00:52<00:12, 6462.84it/s]

 80%|███████▉  | 310584/390021 [00:52<00:12, 6163.32it/s]

 80%|███████▉  | 311227/390021 [00:52<00:12, 6240.35it/s]

 80%|███████▉  | 311874/390021 [00:52<00:12, 6306.94it/s]

 80%|████████  | 312507/390021 [00:53<00:12, 6280.73it/s]

 80%|████████  | 313144/390021 [00:53<00:12, 6307.17it/s]

 80%|████████  | 313776/390021 [00:53<00:12, 6283.96it/s]

 81%|████████  | 314418/390021 [00:53<00:11, 6323.48it/s]

 81%|████████  | 315051/390021 [00:53<00:12, 6086.64it/s]

 81%|████████  | 315691/390021 [00:53<00:12, 6176.22it/s]

 81%|████████  | 316334/390021 [00:53<00:11, 6247.98it/s]

 81%|████████▏ | 316963/390021 [00:53<00:11, 6259.54it/s]

 81%|████████▏ | 317591/390021 [00:53<00:11, 6168.68it/s]

 82%|████████▏ | 318209/390021 [00:53<00:11, 6155.53it/s]

 82%|████████▏ | 318848/390021 [00:54<00:11, 6221.02it/s

OUTPUT LEN - 390021


In [131]:
a[0]

[0.8389467206809907, 0.4260682730717754]

In [75]:
a[1][0].min(1)[1]

tensor([1, 1, 1,  ..., 1, 1, 1])

In [53]:
data.y[train_mask]

tensor([1, 0, 0,  ..., 1, 1, 1])

In [70]:
rr = y[train_mask].tolist()

In [71]:
for i in range(len(rr)):
    if (rr[i] != 1):
        print("Cake?")

Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake

Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake

Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake

Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake

Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake

...

Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake

Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake

Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake

Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake?
Cake

KeyboardInterrupt: 

In [ ]:
@torch.no_grad()
def test():
    model.eval()
    logits, accs = model(), []
    result = []
    for _, mask in data('train_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        result.append(logits[mask])
#       print(logits[mask].max(1))
#        print(mask)
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return [accs, result]

In [42]:
%debug

> /home/user/conda/lib/python3.7/site-packages/torch/nn/functional.py(2218)nll_loss()
   2216                          .format(input.size(0), target.size(0)))
   2217     if dim == 2:
-> 2218         ret = torch._C._nn.nll_loss(input, target, weight, _Reduction.get_enum(reduction), ignore_index)
   2219     elif dim == 4:
   2220         ret = torch._C._nn.nll_loss2d(input, target, weight, _Reduction.get_enum(reduction), ignore_index)

ipdb> input
tensor([[-0.6909, -0.6954],
        [-0.6687, -0.7182],
        [-0.6883, -0.6981],
        ...,
        [-0.6812, -0.7053],
        [-0.6681, -0.7189],
        [-0.6803, -0.7062]], grad_fn=<IndexBackward>)
ipdb> type(input)
<class 'torch.Tensor'>
ipdb> input.dtype
torch.float32
ipdb> target.dtype
torch.float32
ipdb> target
tensor([1., 0., 0.,  ..., 1., 1., 1.])
ipdb> exit


In [149]:
y.shape 

torch.Size([390021])

In [150]:
data.shape 

AttributeError: 'Data' object has no attribute 'shape'

In [15]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
